## Design patterns with first-class functions

### example 1

电子商务网站结算策略

- 用户忠诚度大于 1000， 打 9 折
- 在一个订单中超过 20 样物品，打 9 折
- 订单超过 10 中不同的物品，打 93 折

折扣只能享受一次。



In [11]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
    
class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
            
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
            
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due : {:.2f}>'
        return fmt.format(self.total(), self.due())
        
        
class Promotion:
    
    @abstractmethod
    def discount(self, order):
        '''Return discount as a positive dollar amount'''
        
        
class FidelityPromo(Promotion):
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
    
class BulkPromo(Promotion):
    
    def discount(self, order):
        
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
                
        return discount
    
    
class LargeOrderPromo(Promotion):
    
    def discount(self, order):
        distinct_item = {item.product for item in order.cart}
        if len(distinct_item) >= 10:
            return order.total() * .07
        return 0
    
    

In [4]:
joe = Customer('John joe', 0)
ann = Customer('Ann smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

Order(joe, cart, FidelityPromo())

        

<Order total: 42.00 due : 42.00>

In [5]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due : 39.90>

In [8]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)
              ]

Order(joe, banana_cart, BulkPromo())

<Order total: 30.00 due : 28.50>

In [12]:
long_order = [LineItem(str(item_code), 1, 1.0) 
              for item_code in range(10)]

Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due : 9.30>

### 对例子中的 python 用法做扩展理解

- namedtuple
- abstractmethod
- 跟 abstractmethod 相近的 classmethod/property 装饰器
- 总结这个例子

In [15]:
# namedtuple

from collections import namedtuple

Cus = namedtuple('Cus', 'name age')
Cus('zhouxin', 16)

Cus(name='zhouxin', age=16)

In [16]:
shop = namedtuple('shop', 'cus, code')
cus1 = Cus('zhangqian', 25)
shop1 = shop(cus1, 10000)

In [17]:
shop1

shop(cus=Cus(name='zhangqian', age=25), code=10000)

In [27]:
# abstactmethod

import abc

class Abcls:
    
    __method__ = abc.ABCMeta
    
    @abc.abstractmethod
    def func(self):
        print('in parent')
        raise NotImplementedError("Please Implement this method")

        

class Sub(Abcls):
    
    def __init__(self):
        
        self.me = 'me'
        
    def func(self):
        print('in sub')
        

s = Sub()
s.func()

in sub


#### abstractmethod

看了官方文档和网络上的资料，对与这个装饰器依然摸不着头脑，很简单的一个问题就是，这个有什么用？
感觉加了和不加没有太大区别。

先留着吧

In [30]:
# property 装饰器是将方法变为属性

class A:
    
    @property
    def name(self):
        
        return 'zhouxin'
    
    
a = A()

In [31]:
a.name

'zhouxin'

In [32]:
a.name()

TypeError: 'str' object is not callable

### Function-oriented Strategy


In [47]:
# 沿用上面的 LineItem 类
# 继承上面的 Order 类, 重写其中的 due 方法

class NewOrder(Order):
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    

def fidelity_promo(order):
    
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
            
    return discount

def large_order_promo(order):
    
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0        

In [48]:
NewOrder(ann, cart, fidelity_promo)

<Order total: 42.00 due : 39.90>

In [49]:
NewOrder(joe, long_order, large_order_promo)

<Order total: 10.00 due : 9.30>

In [50]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    
    return max(promo(order) for promo in promos)

In [51]:
NewOrder(joe, cart, best_promo)

<Order total: 42.00 due : 42.00>